In [25]:
import numpy as np
import pysindy as ps
from pysindy.feature_library import PolynomialLibrary, FourierLibrary, CustomLibrary
import matplotlib.pyplot as plt

from generate_data import generate_tracking_data, generate_discrete_tracking_data
from decimal import Decimal

In [ ]:
T = 1e-1
t = np.linspace(0, 20, 201)

x0 = [-0.1, 0.2, -0.1]
x0_val = [0.1, 0.1, 0]

u = lambda t: 0.001*np.sin(t)
u_val = lambda t: 0.001*np.cos(2*t)

x, x_dot = generate_tracking_data(t=t, x0=x0, u=u)
x_val, _ = generate_tracking_data(t=t, x0=x0_val, u=u_val)

xk = generate_discrete_tracking_data(t=t, x0=x0, T=T, u=u(t))
xk_val = generate_discrete_tracking_data(t=t, x0=x0_val, T=T, u=u_val(t))

In [13]:
model = ps.SINDy(
    feature_library=PolynomialLibrary(degree=3), # 2, 1
    # feature_library=FourierLibrary(n_frequencies=1), # 2, 1
    optimizer=ps.STLSQ(threshold=1e-9), # 0.001, 0.01, 0.1
    feature_names=[f'x{i+1}' for i in range(3)]+['u'],
    # discrete_time=True
    )
model.fit(x=x, x_dot=x_dot, u=u(t))
# model.fit(x=xk, u=u(t))
model.print(precision=9)

(x1)' = -0.000001992 1 + -0.877241474 x1 + 0.000021483 x2 + 0.999919807 x3 + -0.216771283 u + 0.468616000 x1^2 + 0.001761271 x1 x2 + -0.088207510 x1 x3 + -0.014449547 x1 u + -0.019077320 x2^2 + 0.000591975 x2 x3 + 0.012952380 x2 u + 0.000134588 x3^2 + 0.001399896 x3 u + -0.023274069 u^2 + 3.843994544 x1^3 + 0.005037290 x1^2 x2 + -0.999944563 x1^2 x3 + 0.257149579 x1^2 u + -0.003210714 x1 x2^2 + 0.000773322 x1 x2 x3 + 0.052563882 x1 x2 u + 0.000331203 x1 x3^2 + 0.006345889 x1 x3 u + 0.351270367 x1 u^2 + 0.000092863 x2^3 + -0.001091315 x2^2 x3 + -0.023668116 x2^2 u + -0.000499270 x2 x3^2 + -0.005154240 x2 x3 u + 0.084648568 x2 u^2 + -0.000063007 x3^3 + 0.001659129 x3^2 u + -0.018279648 x3 u^2
(x2)' = 1.000000000 x3 + -0.000000001 u + 0.000000001 x1 x2 + -0.000000005 x1 u + 0.000000007 x2 u + 0.000000001 x3 u + -0.000000010 u^2 + 0.000000001 x1^3 + 0.000000005 x1^2 u + -0.000000002 x1 x2^2 + 0.000000018 x1 x2 u + 0.000000002 x1 x3 u + 0.000000035 x1 u^2 + -0.000000012 x2^2 u + -0.00000000

In [9]:
t = np.linspace(0, 200, 201)
x0s = np.random.uniform(-0.1, 0.1, (500, 3))

xss = []
x_dots = []
us = []

for x0 in x0s:
    u_val = lambda t: x0[0] / 100. * np.sin(t)
    x, x_dot = generate_tracking_data(t=t, x0=x0, u=u_val)
    xss.append(x)
    x_dots.append(x_dot)
    us.append(u_val(t))

In [14]:
model = ps.SINDy(
    feature_library=PolynomialLibrary(degree=3), # 2, 1
    # feature_library=FourierLibrary(n_frequencies=1), # 2, 1
    optimizer=ps.STLSQ(threshold=1e-9), # 0.001, 0.01, 0.1
    feature_names=[f'x{i+1}' for i in range(3)]+['u']
    )
model.fit(x=xss, x_dot=x_dots, u=us, multiple_trajectories=True)
model.print(precision=9)

(x1)' = -0.877000000 x1 + 1.000000000 x3 + -0.215000000 u + 0.470000000 x1^2 + -0.088000000 x1 x3 + -0.019000000 x2^2 + 3.846000000 x1^3 + -1.000000000 x1^2 x3 + 0.280000000 x1^2 u + 0.470000000 x1 u^2 + 0.630000001 u^3
(x2)' = 1.000000000 x3
(x3)' = -4.208000000 x1 + -0.396000000 x3 + -20.967000000 u + -0.470000000 x1^2 + -0.000000003 x1 u + -0.000000001 x2 u + -0.000000003 x3 u + 0.000000001 u^2 + -3.563999999 x1^3 + 6.265000000 x1^2 u + 0.000000001 x1 x2 x3 + 46.000000000 x1 u^2 + 61.100000000 u^3


In [ ]:
for x_num in range(len(x0)):
    q = 'Biblioteka funkcji & Próg & $\Dot{x}'
    print(f'{q}_{x_num+1}$ & $E_{x_num+1}$ \\\\')
    for i, library in enumerate([FourierLibrary(n_frequencies=2), FourierLibrary(n_frequencies=1), PolynomialLibrary(degree=3), PolynomialLibrary(degree=2), PolynomialLibrary(degree=1)]):
        print(f'\\hline')
        for threshold in range(3):
            threshold = 10**(threshold-10) if i >= 2 else 100*10**(threshold-10)
            # threshold = T * threshold
            name = ['Trygonometryczna (st. 2)', 'Trygonometryczna (st. 1)', 'Wielomiany (st. 3)', 'Wielomiany (st. 2)', 'Liniowa']
            model = ps.SINDy(
                feature_library=library,
                optimizer=ps.STLSQ(threshold=threshold),
                feature_names=[f'x{i+1}' for i in range(len(x0))]+['u'],
                # discrete_time=True
                )
            # model.fit(x=xk, u=u(t))
            model.fit(x=x, x_dot=x_dot, u=u(t))
            try:
                x_sim = model.simulate(x0=x0_val, t=t, u=u_val)
                # x_sim = model.simulate(x0=x0_val, t=201, u=u_val(t))
                mse = ((x_sim - x_val)**2).mean(axis=0)
                E = '%.3E' % Decimal(str(mse[x_num]))
            except:
                E = '\infty'
            coeffs = ' + '.join(['%.3E' % Decimal(str(coeff))+' '+model.get_feature_names()[i] for i, coeff in enumerate(model.coefficients()[x_num]) if abs(model.coefficients()[x_num][i]) > threshold])
            if len(coeffs.split(' + ')) > 2:
                eq = (coeffs.split(' + ')[0] + ' + ' + coeffs.split(' + ')[1] + '\dots').replace(' 1 +', ' +')
            else:
                eq = coeffs
            if len(coeffs) == 0:
                eq = '0,000'
            eq = eq.replace(' 1 +', ' +').replace('.', ',').replace('+ -', '- ').replace('sin', '\sin').replace('cos', '\cos').replace('(1 x1)', '(x_1)').replace('(1 x2)', '(x_2)').replace('x2', 'x_2').replace('(1 x3)', '(x_3)').replace('x3', 'x_3').replace('x1', 'x_1').replace('(1 u)', '(u)')
            thr = '%.0E' % Decimal(str(threshold))
            for pow in range(1, 10):
                eq = eq.replace('E+00', '')
                E = E.replace('E+00', '')
                eq = eq.replace(f'E+0{pow}', f'\cdot 10^{pow}').replace(f'E-0{pow}', '\cdot 10^{'+f'{-pow}'+'}')
                E = E.replace(f'E+0{pow}', f'\cdot 10^{pow}').replace(f'E-0{pow}', '\cdot 10^{'+f'{-pow}'+'}')
                thr = thr.replace(f'1E+0{pow}', f'10^{pow}').replace(f'1E-0{pow}', '10^{'+f'{-pow}'+'}')
            print(f"{name[i]} & ${thr}$ & ${eq}$ & ${E.replace('.', ',')}$ \\\\")
    print('\n\n')

In [ ]:
# x_sim = model.simulate(x0=x0_val, t=t, u=u_val)
# mse = ((x_sim - x_val)**2).mean(axis=0)
# print(f'Błąd średniokwadratowy x1: {mse[0]}, x2: {mse[1]}, x3: {mse[2]}')

In [ ]:
plt.plot(t, x_val[:, 0])
plt.plot(t, x_val[:, 1])
plt.plot(t, x_val[:, 2], 'y')
plt.grid()
plt.legend(["Kąt natarcia", "Kąt nachylenia", "Współczynnik nachylenia"])
plt.xlim(0, max(t))
plt.xlabel("Czas [dni]")
ax1 = plt.gca()
ax2 = ax1.twinx()
ax2.spines['right'].set_color('green')
ax2.yaxis.label.set_color('green')
ax2.tick_params(axis='y', colors='green')
ax2.plot(t, u_val(t), 'g--', alpha=0.4)
ax2.set_ylabel("Sterowanie")
ax2.set_ylim(0, 0.003)
ax1.set_ylabel("Kąt [Rad]")
ax2.set_ylabel("Sterowanie")
plt.show()